In [1]:
# Dennis O'Leary
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
# calculates the totals
totSch = len(school_data['school_name'].unique())
totStu = len(student_data['student_name'])
totBud = school_data['budget'].sum()
totMath = student_data['math_score'].mean()
totRead = student_data['reading_score'].mean()
# uses bins to passes/fails, counts them and calculates percentages
bins = [0, 69, 100]
grpNames = ['fail', 'pass']
readRate = student_data.loc[:, ['student_name', 'reading_score']]
mathRate = student_data.loc[:, ['student_name', 'math_score']]
readRate["reading_score"] = pd.cut(readRate["reading_score"], bins, labels=grpNames)
mathRate["math_score"] = pd.cut(mathRate["math_score"], bins, labels=grpNames)
readRate = readRate.groupby('reading_score')
mathRate = mathRate.groupby('math_score')
readRate = readRate.count()
mathRate = mathRate.count()
readPass = (readRate.iat[1, 0]/readRate['student_name'].sum())*100
mathPass = (mathRate.iat[1, 0]/mathRate['student_name'].sum())*100
overPass = (mathPass+readPass)/2

# create a dataframe to display the output
distSum = {'Total Schools': [totSch],
          'Total Students': [totStu],
          'Total Budget': [totBud],
          'Average Math Score': [totMath],
          'Average Reading Score': [totRead],
          'Passing Math': [mathPass],
          'Passing Reading': [readPass],
          'Overall Passing Rate': [overPass]}
distSum = pd.DataFrame(distSum)
# format columns
distSum['Total Budget'] = distSum['Total Budget'].map("${:.2f}".format)
distSum['Average Math Score'] = distSum['Average Math Score'].map("{:.2f}".format)
distSum['Average Reading Score'] = distSum['Average Reading Score'].map("{:.2f}".format)
distSum['Passing Math'] = distSum['Passing Math'].map("{:.2f}%".format)
distSum['Passing Reading'] = distSum['Passing Reading'].map("{:.2f}%".format)
distSum['Overall Passing Rate'] = distSum['Overall Passing Rate'].map("{:.2f}%".format)

distSum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,15,39170,$24649428.00,78.99,81.88,74.98%,85.81%,80.39%


In [4]:
# creates a dataframe grouped by school and type using .mean() which gives average scores and the budget and total students
topSchools = school_data_complete.loc[:, ['school_name', 'type', 'size', 'budget', 'math_score', 'reading_score']]
topSchools = topSchools.rename(columns={'school_name': 'School Name', 'type': 'School Type'})
topSchGrp = topSchools.groupby(['School Name', 'School Type'])
topSchGrp = topSchGrp.mean()
# adds a column for the Per Student Budget
topSchGrp['Per Student Budget'] = topSchGrp['budget']/topSchGrp['size']
# uses bins to passes/fails for math and reading
bins = [0, 69, 100]
grpNames = ['fail', 'pass']
readRate = school_data_complete.loc[:, ['school_name', 'student_name', 'reading_score']]
mathRate = school_data_complete.loc[:, ['school_name', 'student_name', 'math_score']]
readRate["reading_score"] = pd.cut(readRate["reading_score"], bins, labels=grpNames)
mathRate["math_score"] = pd.cut(mathRate["math_score"], bins, labels=grpNames)
readRate = readRate.groupby(['school_name','reading_score'])
mathRate = mathRate.groupby(['school_name','math_score'])
readRate = readRate.count()
mathRate = mathRate.count()
mathPass = []
mathFail = []
readPass = []
readFail = []
i = 1
# loops through math and reading pass/fail bins for each school 
while i < 30:
    mathPass.append(mathRate.iat[i, 0])
    mathFail.append(mathRate.iat[i-1, 0])
    readPass.append(readRate.iat[i, 0])
    readFail.append(readRate.iat[i-1, 0])
    i = i + 2
# calculates math pass% for each school and adds it as a column in our dataframe
topSchGrp['% Passing Math'] = mathPass
topSchGrp['mathFail'] = mathFail
topSchGrp['% Passing Math'] = (topSchGrp['% Passing Math']/(topSchGrp['% Passing Math']+topSchGrp['mathFail']))*100
del topSchGrp['mathFail']
# calculates reading pass% for each school and adds it as a column in our dataframe
topSchGrp['% Passing Reading'] = readPass
topSchGrp['readFail'] = readFail
topSchGrp['% Passing Reading'] = (topSchGrp['% Passing Reading']/(topSchGrp['% Passing Reading']+topSchGrp['readFail']))*100
del topSchGrp['readFail']
# calculates overall pass% for each school and adds it as a column in our dataframe
topSchGrp['% Overall Passing Rate'] = (topSchGrp['% Passing Math']+topSchGrp['% Passing Reading'])/2
# renames, sorts, and formats our columns
topSchGrp = topSchGrp.rename(columns={'size': 'Total Students',
                                        'budget': 'Total School Budget',
                                        'math_score': 'Average Math Score',
                                        'reading_score': 'Average Reading Score'})
topSchGrp = topSchGrp.loc[:,['Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score',
                            'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']] 
topSchGrp = topSchGrp.sort_values(by=['% Overall Passing Rate'], ascending=False)
botSchGrp = topSchGrp.sort_values(by=['% Overall Passing Rate'], ascending=True)
# creates unformated dataframe with indexes reset for later use
unformSch = topSchGrp.reset_index()
topSchGrp['Total Students'] = topSchGrp['Total Students'].map("{:.0f}".format)
topSchGrp['Total School Budget'] = topSchGrp['Total School Budget'].map("${:.2f}".format)
topSchGrp['Average Math Score'] = topSchGrp['Average Math Score'].map("{:.2f}".format)
topSchGrp['Average Reading Score'] = topSchGrp['Average Reading Score'].map("{:.2f}".format)    
topSchGrp['% Passing Math'] = topSchGrp['% Passing Math'].map("{:.2f}%".format)
topSchGrp['% Passing Reading'] = topSchGrp['% Passing Reading'].map("{:.2f}%".format)
topSchGrp['% Overall Passing Rate'] = topSchGrp['% Overall Passing Rate'].map("{:.2f}%".format)
topSchGrp['Per Student Budget'] = topSchGrp['Per Student Budget'].map("${:.2f}".format)

topSchGrp.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,School Type,,,,,,,,
Cabrera High School,Charter,1858,$1081356.00,$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,$1043130.00,$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,$585858.00,$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,$917500.00,$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,$1319574.00,$578.00,83.27,83.99,93.87%,96.54%,95.20%
